# Downstream application package

Invoke the downstream application package that calculates the mean.

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-workflows/downstream/



## Setup

In [1]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.3.0/app-water-bodies-occurrence.0.3.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies-occurrence.cwl 2> /dev/null

## Run the downstream application package 

Inspect and use `cwltool` to run the downstream application package CWL Workflow:


In [2]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies-occurrence.cwl | yq e '.$graph[0]' -

class: Workflow
id: water-bodies-occurrence
label: Water bodies occurrence
doc: Water bodies occurrence based on NDWI and otsu threshold
requirements: []
inputs:
  zarr-stac-catalog:
    type: Directory
    doc: Input STAC catalog with datacube
    label: STAC catalog
outputs:
  stac-catalog:
    type: Directory
    outputSource:
      - step_occurrence/stac-catalog
    doc: Output STAC catalog with water bodies occurrence
    label: STAC catalog
steps:
  step_occurrence:
    run: "#occurrence"
    in:
      zarr-stac-catalog: zarr-stac-catalog
    out:
      - stac-catalog


Create the job order using the outputs of notebook "2-CWl-Workflows/1-Zarr-Cloud-native-format" 

In [5]:
zarr_catalog_path=$(cat zarr-cloud-native-results.json | jq -r '.zarr_stac_catalog.path' -) 
echo $zarr_catalog_path

/workspace/zarr-cloud-native-format/runs/gaaevbi7


In [6]:
cat <<EOF > downstream-params.yaml
zarr-stac-catalog: 
  class: Directory
  path: ${zarr_catalog_path}
EOF

cat downstream-params.yaml | yq .

zarr-stac-catalog:
  class: Directory
  path: /workspace/zarr-cloud-native-format/runs/gaaevbi7


Use `cwltool` to run the CWL Workflow

In [ ]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies-occurrence.cwl#water-bodies-occurrence \
    downstream-params.yaml > downstream-results.json 2> downstream.log

Let's look at the content of the stderr:

In [ ]:
cat downstream.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO Resolved '/workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies-occurrence.cwl#water-bodies-occurrence' to 'file:///workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies-occurrence.cwl#water-bodies-occurrence'
INFO [workflow ] start
INFO [workflow ] starting step step_occurrence
INFO [step step_occurrence] start
Error: no such object: "ghcr.io/eoap/zarr-cloud-native-format/occurrence@sha256:256f8672ff74f41b97ab307162b09e07412eddd1d9658a07106379597a7c9fad"
INFO ['podman', 'pull', 'ghcr.io/eoap/zarr-cloud-native-format/occurrence@sha256:256f8672ff74f41b97ab307162b09e07412eddd1d9658a07106379597a7c9fad']
Trying to pull ghcr.io/eoap/zarr-cloud-native-format/occurrence@sha256:256f8672ff74f41b97ab307162b09e07412eddd1d9658a07106379597a7c9fad...
Getting image source signatures
Copying blob sha256:ddee35dfe011992ed9d752dc61e7dc4377fda83f63e03c0e193a4a4e6714cc2b
Copying blob sha256:8a7a13810b7d60799d98f4a85010c534

Let's inspect the stdout produced. The `output` block with the id `stac-catalog` of the CWL description is a directory containing the STAC catalog:

In [7]:
cat downstream-results.json | jq '.["stac-catalog"].path' -

"/workspace/zarr-cloud-native-format/runs/_9l_9a__"


List the STAC Catalog content:

In [8]:
stac describe $(cat downstream-results.json | jq -r '.["stac-catalog"].path' - )/catalog.json

* <Catalog id=catalog>
  * <Item id=occurrence>


In [10]:
tree $(cat downstream-results.json | jq -r '.["stac-catalog"].path' - )

/workspace/zarr-cloud-native-format/runs/_9l_9a__
├── catalog.json
└── occurrence
    ├── occurrence.json
    └── water_bodies_mean.tif

2 directories, 3 files
